In [1]:
#import required libraries
import heapq
import numpy as np
import scipy.stats as sts

In [2]:
class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
    
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        # Return the time of the next event. The `now` attribute of this class
        # contain the time of the last event that was run.
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        # Print out diagnostic information about the events in the schedule.
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

In [3]:
#Arrival rate with an exponential distribution of a rate
#add arrival to the queue
#check if the server is free
#if free, start the service
#otherwise do nothing
#when serving, remove one from queue and add another event for start serving
#constant service rate

class Queue:
    '''
    Creates a queue for customers and handles their service.
    Parameters:
    -----------
    service_rate : float
        Rate at which the server can serve the customers
    '''
    def __init__(self, service_rate):
        self.service_rate = service_rate
        self.service_time = 1/service_rate #fixed service time
        self.queue_len = 0 #start with 0 people in queue
        self.people_serving = 0 #start with 0 people in bus
        self.visit_counter = 0 #start at 0 visitors
        self.service_counter = 0 #start with 0 people serviced
    
    def arrival_process(self, schedule):
        '''
        Adds an arrival to the queue and decides if server is available.
        Parameters
        ----------
        schedule : Schedule
            A Schedule object which has a heap of all events.
        '''
        #increase the queue length
        self.queue_len += 1
        self.visit_counter += 1
        print(f"New arrival no. {self.visit_counter}, checking if server is empty...")

        #check if server is free
        if self.people_serving < 1:
            #add event to start serving immediately
            schedule.add_event_after(0, self.start_serve)
        
        else:
            print("Server not free, new arrival starts waiting in the queue")
    
    def start_serve(self, schedule):
        '''
        Starts the service when the server is available.
        '''
        #remove person from queue and add to currently serving
        self.queue_len -= 1
        self.people_serving += 1
        self.service_counter += 1
        print(f"Server available, starting service for visitor number {self.service_counter}")

        #add event to stop the service
        schedule.add_event_after(self.service_time, self.stop_serve)

    def stop_serve(self, schedule):
        '''
        Stops the service and checks for waiting customers.
        '''
        #remove person from the system
        self.people_serving -= 1
        print(f"Finished service for visitor number {self.service_counter}")
        print("Checking queue for more visitors...")

        #check if still people in queue
        if self.queue_len > 0:
            schedule.add_event_after(0, self.start_serve)
        
        else:
            print("Queue is empty, server unused for now.")


class BusSystem:
    '''
    Create a system for simulation of a Bus with a single person capacity.
    '''
    def __init__(self, arrival_rate, service_rate):
        self.arrival_rate = arrival_rate
        self.queue = Queue(service_rate)             #create the queue
        self.arrival_dist = sts.expon(scale = 1/self.arrival_rate)
    
    def arrival(self, schedule):
        #go through the arrival process for each arrival event
        self.queue.arrival_process(schedule)

        #calculate the arrival time
        arrival_time = self.arrival_dist.rvs()

        #add another arrival event after dealing with the previous arrival
        #print(f"Adding another arrival event at {schedule.now() + arrival_time}")
        schedule.add_event_after(arrival_time, self.arrival)

    def start(self, schedule):
        #add first arrival event
        print("Starting the simulation")
        schedule.add_event_after(self.arrival_dist.rvs(), self.arrival)

def run_simulation(arrival_rate, service_rate, run_until):
    '''
    Run the M/D/1 simulation based on Schedule and Event classes. 
    Parameters
    ----------
    arrival_rate : float
        Rate of arrivals of customers
    service_rate : float
        Service rate for customers
    '''
    bus_schedule = Schedule()
    bus_system = BusSystem(arrival_rate, service_rate)
    bus_system.start(bus_schedule)
    while bus_schedule.now <= run_until:
        bus_schedule.run_next_event()

In [ ]:
#set the simulation parameters
arrival_rate = 2
service_rate = 2
run_until = 3

run_simulation(arrival_rate, service_rate, run_until)

Starting the simulation
New arrival no. 1, checking if server is empty...
Server available, starting service for visitor number 1
Finished service for visitor number 1
Checking queue for more visitors...
Queue is empty, server unused for now.
New arrival no. 2, checking if server is empty...
Server available, starting service for visitor number 2
New arrival no. 3, checking if server is empty...
Server not free, new arrival starts waiting in the queue
Finished service for visitor number 2
Checking queue for more visitors...
Server available, starting service for visitor number 3
Finished service for visitor number 3
Checking queue for more visitors...
Queue is empty, server unused for now.


In [7]:
arrival_rate = 1.2
arrival_distribution = sts.expon(scale=1/arrival_rate)

a = [1, 2, 3]
b = [1, 2, 3]

**#PythonImplementation**: My simulation is working and I have used heaps as I was supposed to. I have also organized my code using Object-oriented programming. I have included helpful print statements that helps understand what the code is doing for small runtimes and it can be commented out for larger runtimes. 

**#CodeReadability**: I have used appropriate variable names that make sense. I have mostly made sure not to use magic numbers or assumptions. Moreover, I have added appropriate docstrings and comments as needed. I have also added helpful print commands for clearer understand of flow of algorithm.